In [1]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

In [2]:
!pip install tensorflow

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers.legacy import SGD


In [4]:
lemmatizer = WordNetLemmatizer()

intents=json.loads(open('intents.json').read())

words= []
classes=[]
documents=[]
ignore_letters=['?', '!', '.', ',']

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list=nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append(((word_list), intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
print(documents)

[(['hello'], 'greetings'), (['hey'], 'greetings'), (['good', 'day'], 'greetings'), (['greetings'], 'greetings'), (['what', "'s", 'up', '?'], 'greetings'), (['how', 'you', 'doing', '?'], 'greetings'), (['bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['good', 'day'], 'goodbye'), (['greetings'], 'goodbye'), (['Need', 'to', 'go'], 'goodbye'), (['I', 'am', 'leaving'], 'goodbye'), (['what', 'is', 'your', 'name', '?'], 'name'), (['Your', 'good', 'name', '?'], 'name'), (['What', 'should', 'I', 'call', 'you', '?'], 'name'), (['Name', '?'], 'name'), (['how', 'old', '?'], 'age'), (['how', 'old', 'is', 'Jyotsna', '?'], 'age'), (['what', 'is', 'your', 'age', '?'], 'age'), (['Need', 'anything', 'to', 'drink', '?'], 'drink'), (['what', 'you', 'drink', '?'], 'drink'), (['what', 'would', 'you', 'like', 'to', 'drink', '?'], 'drink'), (['Do', 'you', 'drink', '?'], 'drink'), (['What', 'share', 'should', 'I', 'bought', '?'], 'share'), (['Good', 'shares'], 'share'), (['high', 'value', 'stock', '

In [7]:
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))
print(words)

["'s", 'Do', 'Good', 'I', 'Jyotsna', 'Name', 'Need', 'See', 'What', 'Your', 'age', 'am', 'anything', 'best', 'bought', 'bye', 'call', 'day', 'doing', 'drink', 'go', 'good', 'greeting', 'hello', 'hey', 'high', 'how', 'is', 'later', 'leaving', 'like', 'name', 'old', 'share', 'should', 'stock', 'to', 'up', 'value', 'what', 'would', 'you', 'your']


In [8]:
classes=sorted(set(classes))

pickle.dump(words, open("words.pk1", "wb"))
pickle.dump(classes, open("classes.pk1", "wb"))

In [9]:
#prepocessing the data
training=[]
output_empty=[0]*len(classes)

for document in documents:
    bag=[]
    word_patterns=document[0]
    word_patterns=[lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
        
    output_row=list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag, output_row])

In [10]:
random.shuffle(training)
training=np.array(training)

C:\Users\user\AppData\Local\Temp\ipykernel_2568\4111924452.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


In [11]:
#training the neural network
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [12]:
model=Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd=SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist=model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save("ChatBot_model.h5", hist)
print("Done")

C:\Users\user\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/200
6/6 [==============================] - 1s 4ms/step - loss: 1.9137 - accuracy: 0.0370
Epoch 2/200
6/6 [==============================] - 0s 3ms/step - loss: 1.8183 - accuracy: 0.1852
Epoch 3/200
6/6 [==============================] - 0s 3ms/step - loss: 1.7780 - accuracy: 0.1111
Epoch 4/200
6/6 [==============================] - 0s 3ms/step - loss: 1.7182 - accuracy: 0.3333
Epoch 5/200
6/6 [==============================] - 0s 3ms/step - loss: 1.6804 - accuracy: 0.3333
Epoch 6/200
6/6 [==============================] - 0s 3ms/step - loss: 1.6091 - accuracy: 0.3704
Epoch 7/200
6/6 [==============================] - 0s 3ms/step - loss: 1.6001 - accuracy: 0.4444
Epoch 8/200
6/6 [==============================] - 0s 3ms/step - loss: 1.5637 - accuracy: 0.2963
Epoch 9/200
6/6 [==============================] - 0s 4ms/step - loss: 1.5021 - accuracy: 0.4815
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 1.4656 - accuracy: 0.4815
Epoch 11/200
6/6 [===========